In [1]:
# Get imports from imports.py
# Will try to pull imports from ./../modules/
import sys
sys.path.append('./../run_pipeline/')
from imports import *

In [2]:
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c'
hdu_hst = fits.open(f"{root_dir}/hst_contsub/ngc628c_hst_ha_as2.fits")[0]
hsu_muse = fits.open(f"{root_dir}/hst_contsub/ngc628c_muse_ha_as2.fits")[0]

In [6]:
from astropy.io import fits
import astropy.wcs as wcs
import astropy.units as u
from astropy.convolution import convolve_fft
from astropy.stats import mad_std
import numpy as np
from radio_beam import Beam
from reproject import reproject_interp
from matplotlib import pyplot as plt
from scipy.ndimage import binary_dilation
from astropy.modeling import models, fitting
from astropy.table import Table, vstack 
from glob import glob 
from synphot import SpectralElement, units
from matplotlib.ticker import (MultipleLocator)
import os
import warnings 
warnings.filterwarnings('ignore')


def get_smooth(hdu, initial_resolution, desired_resolution):
    
    # Create a WCS object from the input HDU header
    wcs_ = wcs.WCS(hdu.header)

    # Calculate the pixel scale in degrees
    pixscale = wcs.utils.proj_plane_pixel_area(wcs_.celestial) ** 0.5 * u.deg
    print(f"[INFO] Pixel scale: {pixscale.to('arcsec'):.2f} arcsec")

    # Define the initial and desired beams
    initial_beam = Beam(initial_resolution)
    desired_beam = Beam(desired_resolution)

    print(f"[INFO] Initial Resolution: {initial_resolution.to('arcsec'):.2f} arcsec")
    print(f"[INFO] Desired Resolution: {desired_resolution.to('arcsec'):.2f} arcsec")
    
    # Create the convolution kernel
    convolution_beam = (desired_resolution.to('arcsec')**2 - initial_resolution.to('arcsec')**2)**0.5
    convolution_kernel = desired_beam.deconvolve(initial_beam).as_kernel(pixscale)
    print(f"[INFO] Convolution kernel: {convolution_beam.to('arcsec'):.2f} arcsec")

    # Convolve the image with the kernel to smooth it
    print("[INFO] Performing image convolution...")
    smoothed_data = convolve_fft(hdu.data, convolution_kernel, preserve_nan=True, allow_huge=True)
    print("[INFO] Image convolution complete.")

    output_hdu = fits.PrimaryHDU(np.array(smoothed_data, dtype=np.float32), hdu.header)

    print("[INFO] Smoothing process completed.")
    return(output_hdu)

def get_regrid(hdu_input, hdu_template, output_filename=None, conserve_flux=True, order='bilinear'):

    print("[INFO] Reprojecting the input image to match the template WCS...")

    # Extract the WCS information from the input and template headers
    header_input = hdu_input.header
    header_template = hdu_template.header
    wcs_input = wcs.WCS(header_input)
    wcs_template = wcs.WCS(header_template)

    # Calculate the pixel scale for input and template images
    pixscale_input = wcs.utils.proj_plane_pixel_area(wcs_input.celestial)
    pixscale_template = wcs.utils.proj_plane_pixel_area(wcs_template.celestial)

    # Reproject the input image to match the template WCS
    print("[INFO] Performing image reprojection...")
    data_output = reproject_interp(hdu_input, header_template, order=order)[0]
    print("[INFO] Image reprojection complete.")

    hdu_output = fits.PrimaryHDU(data_output, header_template)

    if conserve_flux:
        # Scale the output data to conserve flux 
        print(f"[INFO] Scaling the output data to conserve flux with factor {(pixscale_template / pixscale_input):.2f}")
        hdu_output.data = hdu_output.data * (pixscale_template / pixscale_input)
        hdu_output.data = np.array(hdu_output.data, dtype=np.float32)
        print("[INFO] Flux scaling complete.")

    print("[INFO] Reprojection process completed.")
    return(hdu_output)

# hdu_hst_sm = get_smooth(hdu_hst, 0.07 * u.arcsec, 0.92 * u.arcsec)
hdu_hst_smre = get_regrid(hdu_hst_sm, hsu_muse, conserve_flux=False)

[INFO] Reprojecting the input image to match the template WCS...
[INFO] Performing image reprojection...
[INFO] Image reprojection complete.
[INFO] Reprojection process completed.


In [7]:
hdu_hst_smre.writeto(f"{root_dir}/hst_contsub/ngc628c_hst_ha_as2_smreMUSE.fits", overwrite=True)